In [6]:
# Time packages
import cftime, datetime, time
# Numerical analysis packages
import numpy as np, random, scipy, numba
# Local data storage packages
import functools, os, pickle, collections, sys, importlib
# Data structure packages
import pandas as pd, xarray as xr, nc_time_axis
xr.set_options(keep_attrs=True)
# Visualization tools
import cartopy, cartopy.crs as ccrs, matplotlib, matplotlib.pyplot as plt
# Local imports
import accessor, composite, composite_snapshots, derived, ibtracs, utilities, socket, visualization, tc_analysis, tc_processing, track_TCs, TC_tracker

from multiprocessing import Pool

importlib.reload(TC_tracker);
importlib.reload(track_TCs);
importlib.reload(ibtracs);

#### Notes
- IBTrACS data provides observational data that is more intense than ERA5 outputs

In [7]:
def access_IBTrACS_tracks(basin_name: str,
                          date_range: tuple[str, str],
                          intensity_parameter: str,
                          intensity_range: tuple[int, int]) -> pd.DataFrame:

    ''' Access IBTrACS data for a given basin, date range, and intensity bin. '''
    
    track_data = ibtracs.main(basin_name=basin_name,
                              date_range=date_range,
                              intensity_parameter=intensity_parameter,
                              intensity_range=intensity_range)
    
    return track_data.sort_values('time')

In [8]:
def access_observation_data(dataset_name: str, 
                            date_range: tuple[int, int],
                            dirname: str|None=None) -> xr.Dataset:

    ''' Obtain data for observational files within a given year range. '''

    # Load a data directory containing MERRA files if one is not already given
    if not dirname:
        observation_dirnames = {'CERES': '/scratch/gpfs/GEOCLIM/gr7610/tiger3/reference/datasets/CERES',
                                'MERRA': '/scratch/gpfs/GEOCLIM/gr7610/tiger3/reference/datasets/MERRA'}
        assert dataset_name in observation_dirnames.keys(), f'Attempting to find directory for dataset {dataset_name}, but one is not available.'
        dirname = observation_dirnames[dataset_name]
    # Define lambda function to search for file dates
    # Throw error if dataset not found
    if dataset_name == 'MERRA':
        find_date = lambda f: pd.to_datetime(f.split('.nc4')[0].split('.')[-1])
    elif dataset_name == 'CERES':
        find_date = lambda f: pd.to_datetime(f.split('Subset_')[-1].split('.')[0].split('-')[0])
    else:
        assert dataset_name in ['MERRA', 'CERES'], f'[access()] Cannot process datetimes for dataset {dataset_name}. Please define how to process this dataset or enter another dataset name.'
    # List files in the directory that are netCDF files and are within the date range
    filenames = [filename for filename in os.listdir(dirname) if
                 '.nc' in filename and
                  min(date_range) <= find_date(filename) <= max(date_range)]
    # Generaet full pathnames
    pathnames = [os.path.join(dirname, filename) for filename in filenames]
    # Load the dataset
    dataset = xr.open_mfdataset(pathnames, engine='netcdf4')

    # Adjust coordinates for MERRA: map (-180, 180) to (0, 360)
    if dataset_name == 'MERRA':
        dataset['lon'] = dataset['lon'].where(dataset['lon'] > 0, dataset['lon'] + 360)
        dataset = dataset.sortby('lon')
        

    return dataset

In [9]:
def rename_observation_fields(dataset: xr.Dataset,
                              dataset_name: str) -> xr.Dataset:

    ''' Rename fields to match GFDL GCM output data variable names. '''
    
    field_names = {'MERRA': {'SWTDN': 'swdn_toa',
                             'SWTNT': 'swabs_toa',
                             'SWGDN': 'swdn_sfc',
                             'SWGNT': 'swnet_sfc',
                             'LWGAB': 'lwdn_sfc',
                             'LWGEM': 'lwup_sfc',
                             'LWGNT': 'lwnet_sfc',
                             'LWTUP': 'olr',
                             'lon': 'longitude',
                             'lat': 'latitude'},
                   'CERES': {'toa_solar_all_1h': 'swdn_toa',
                             'adj_atmos_sw_up_all_toa_1h': 'swup_toa',
                             'toa_sw_all_1h': 'swabs_toa',
                             'toa_net_all_1h': 'netrad_toa',
                             'adj_atmos_sw_down_all_surface_1h': 'swdn_sfc',
                             'adj_atmos_sw_up_all_surface_1h': 'swup_sfc',
                             'adj_atmos_lw_down_all_surface_1h': 'lwdn_sfc',
                             'adj_atmos_lw_up_all_surface_1h': 'lwup_sfc',
                             'toa_lw_all_1h': 'olr',
                             'lon': 'longitude',
                             'lat': 'latitude'}}

    assert dataset_name in field_names.keys(), f'Field names not found for dataset {dataset_name}.'

    # Rename the data variables
    dataset = dataset.rename(field_names[dataset_name])
    
    return dataset

In [10]:
def derive_fields(dataset: xr.Dataset,
                  dataset_name: str) -> xr.Dataset:

    ''' Derive fields from input data. '''

    # Define fields to derive and the fields required to calculate them
    derived_field_definitions = {'MERRA': {'netrad_toa': ['swabs_toa', 'olr'],
                                           'swup_sfc': ['swabs_sfc', 'swdn_sfc'],
                                           'swdn_sfc': ['swabs_toa', 'swdn_toa'],
                                           'net_lw': ['lwup_sfc', 'lwdn_sfc', 'olr'],
                                           'net_sw': ['swup_sfc', 'swdn_sfc', 'swabs_toa']},
                                'CERES': {'net_lw': ['lwup_sfc', 'lwdn_sfc', 'olr'],
                                          'net_sw': ['swup_sfc', 'swdn_sfc', 'swabs_toa']}}

    assert dataset_name in derived_field_definitions.keys(), f'Field names not found for dataset {dataset_name}.'
    
    # Ensure required fields are loaded
    for derived_field, required_fields in derived_field_definitions[dataset_name].items():
        assert [required_field in dataset.data_vars for required_field in required_fields], f'Fields {required_fields} must be in dataset to derived {derived_field}.'

    if dataset_name == 'MERRA':
        # Derive net radiation (net downward shortwave at TOA - upwelling longwave at TOA)
        dataset['netrad_toa'] = dataset['swabs_toa'] - dataset['olr']
        # Derive upwards shortwave flux at surface. Flip sign such that upwards values are positive.
        dataset['swup_sfc'] = -1 * (dataset['swnet_sfc'] - dataset['swdn_sfc'])
        # Derive upwards shortwave flux at surface. Flip sign such that upwards values are positive.
        dataset['swup_toa'] = -1 * (dataset['swabs_toa'] - dataset['swdn_toa'])
        # Derive column net longwave flux, with positive equaling heating into the atmosphere.
        dataset['net_lw'] = dataset['lwup_sfc'] - dataset['lwdn_sfc'] - dataset['olr']
        # Derive column net shortwave flux, with positive equaling heating into the atmosphere.
        dataset['net_sw'] = dataset['swup_sfc'] - dataset['swdn_sfc'] + dataset['swabs_toa']
    elif dataset_name == 'CERES':
        # Derive column net longwave flux, with positive equaling heating into the atmosphere.
        dataset['net_lw'] = dataset['lwup_sfc'] - dataset['lwdn_sfc'] - dataset['olr']
        # Derive column net shortwave flux, with positive equaling heating into the atmosphere.
        dataset['net_sw'] = dataset['swup_sfc'] - dataset['swdn_sfc'] + dataset['swabs_toa']

    return dataset

In [11]:
def adjust_timestamps(dataset: xr.Dataset) -> xr.Dataset:

    ''' Adjust timestamps to `pandas` format. '''
    
    # Ensure timestamps are selected to the nearest hour
    dataset['time'] = [datetime.datetime(year=timestamp.dt.year.item(),
                                         month=timestamp.dt.month.item(),
                                         day=timestamp.dt.day.item(),
                                         hour=timestamp.dt.hour.item()) for timestamp in dataset.time]
    # Convert to Pandas objects
    dataset['time'] = [pd.to_datetime(timestamp) for timestamp in dataset.time.values]

    return dataset

In [12]:
def adjust_grid(dataset: xr.Dataset,
                target_resolution: float|None=None) -> xr.Dataset:

    ''' Make grid square if not already square by interpolation. '''
    # Get differential in grid spacing in each direction
    zonal_grid_spacing = np.unique(dataset['longitude'].diff('longitude').values)   
    meridional_grid_spacing = np.unique(dataset['latitude'].diff('latitude').values)
    # Check if differentials are equal for all cells
    check_grid_spacing = np.all(np.isclose(zonal_grid_spacing, zonal_grid_spacing[0])) 
    check_meridional_grid_spacing = np.all(np.isclose(meridional_grid_spacing, meridional_grid_spacing[0]))
    # Ensure grids are regularly-spaced
    assert check_grid_spacing & check_meridional_grid_spacing, 'Grid is irregularly spaced.'
    
    # Get smaller of two resolutions, if target not provided
    if not target_resolution:
        target_resolution = min(zonal_grid_spacing[0], meridional_grid_spacing[0])
        
    # Define basis vectors for interpolation
    zonal_basis_vector = np.arange(0, 360, target_resolution)
    meridional_basis_vector = np.arange(-90, 90, target_resolution)

    # Interpolate grid
    dataset = dataset.interp(longitude=zonal_basis_vector).interp(latitude=meridional_basis_vector)

    return dataset

In [13]:
def align_timestamps(TC_track_dataset: pd.DataFrame,
                     observation_dataset: xr.Dataset) -> list:
    
    # Select random timestamps from each dataset to ensure they are the same
    # Assume all timestamps within a dataset have the same timestamp type
    random_TC_timestamp = random.choice(TC_track_dataset.time.values)
    random_observation_timestamp = random.choice(observation_dataset.time.values)

    # Ensure timestamps are identically-typed
    check_timestamp_formats = type(random_TC_timestamp) == type(random_observation_timestamp)
    assert check_timestamp_formats, f'Timestamp types between IBTrACS and reanalysis require alignment. IBTrACS timestamp: {random_TC_timestamp}; observational timestamp: {random_observation_timestamp}'
    
    # Iterate through storm timestamps to make sure they are in the reanalysis data
    observation_TC_timestamps = [TC_timestamp for TC_timestamp in TC_track_dataset.time.values if
                                 TC_timestamp in observation_dataset.time.values]

    return observation_TC_timestamps

In [14]:
def observation_GFDL_compatibility_adjustments(TC_observation_dataset: xr.Dataset) -> xr.Dataset:

    ''' Perform adjustments so reanalysis data can use the same conventions as GFDL output data. '''

    # Rename spatial basis vector coordinate names
    TC_observation_dataset = TC_observation_dataset.rename({'longitude': 'grid_xt', 'latitude': 'grid_yt'})

    # Perform deep copy for coordinate value modification
    TC_observation_dataset_reformatted = TC_observation_dataset.copy(deep=True)

    # Adjust timestamp format to cftime on the xArray Dataset.
    pd_timestamps = pd.to_datetime(TC_observation_dataset.time) # convert to Pandas objects for easier indexing
    cftime_timestamps = [cftime.datetime(year=timestamp.year, 
                                         month=timestamp.month,
                                         day=timestamp.day,
                                         hour=timestamp.hour,
                                         calendar='julian') for timestamp in pd_timestamps]
    TC_observation_dataset_reformatted['time'] = cftime_timestamps
    assert 'cftime' in str(type(TC_observation_dataset_reformatted['time'].values[0])), f'[observation_GFDL_compatibility_adjustments()] Timestamp is not a cftime object.' 

    return TC_observation_dataset_reformatted

In [15]:
def get_TC_coordinates(TC_track_dataset: pd.DataFrame,
                       observation_dataset: xr.Dataset,
                       observation_TC_timestamps: list,
                       observation_resolution: float) -> dict:

    interval_round = lambda x, y: y * round(x / y) # round coordinates to nearest dataset coordinates
    
    # Initialize dictionary for storm track coordinates
    TC_track_coordinates = {}
    # Construct dictionary for coordinates pertaining to each storm timestamp
    for observation_TC_timestamp in observation_TC_timestamps:
        # Obtain longitude and latitude for each timestamp
        TC_track_longitude = TC_track_dataset['center_lon'].loc[TC_track_dataset['time'] == observation_TC_timestamp]
        TC_track_latitude = TC_track_dataset['center_lat'].loc[TC_track_dataset['time'] == observation_TC_timestamp]
        # Round coordinates to align with dataset coordinate system and resolution
        TC_track_coordinates[observation_TC_timestamp] = {'lon': interval_round(TC_track_longitude.item(), observation_resolution),
                                                          'lat': interval_round(TC_track_latitude.item(), observation_resolution)}

    return TC_track_coordinates

In [16]:
def observation_grid_redefinition(TC_track_coordinates: dict,
                                  observation_resolution: float,
                                  coarsen_factor: int,
                                  observation_TC_window_size: int|float):

    ''' Generate a consistent grid for reanalysis data to allow for all timestamps to be interpolated to the same grid. '''

    # Coarsening factor
    interpolation_resolution = observation_resolution * coarsen_factor
    
    # Define storm spatial extents for future interpolation
    minimum_longitude = np.min([entry['lon'] for entry in TC_track_coordinates.values()])
    minimum_latitude = np.min([entry['lat'] for entry in TC_track_coordinates.values()])
    maximum_longitude = np.max([entry['lon'] for entry in TC_track_coordinates.values()])
    maximum_latitude = np.max([entry['lat'] for entry in TC_track_coordinates.values()])
    
    # Define basis vectors for data interpolation
    # Subtract and add window sizes to minima and maxima, respectively, to capture full desired extent
    zonal_basis_vector = np.arange(minimum_longitude - observation_TC_window_size, 
                                   maximum_longitude + observation_TC_window_size, interpolation_resolution)
    meridional_basis_vector = np.arange(minimum_latitude - observation_TC_window_size, 
                                        maximum_latitude + observation_TC_window_size, interpolation_resolution)

    return zonal_basis_vector, meridional_basis_vector

In [17]:
def load_observation_TC_timestamp(TC_track_coordinates: dict,
                                  observation_TC_window_size: int | float,
                                  observation_resolution: float,
                                  observation_dataset: xr.Dataset,
                                  zonal_basis_vector: np.array,
                                  meridional_basis_vector: np.array,
                                  TC_timestamp: cftime.datetime):

    ''' 
    Method to link track data and reanalysis data for a single timestamp. 
    This is compartmentalized to allow for straightforward parallelization.
    '''

    # Define reanalysis dataset coordinate names
    grid_xt = 'longitude'
    grid_yt = 'latitude'
    
    # Initialize container dictionaries
    observation_TC_container = {}
    observation_TC_extent = {}
    observation_TC_extent[TC_timestamp] = {}
    
    # Generate trimming window extents for each timestamp.
    # Window extents are defined as: 
    # 'grid_xt' = (longitude - window_extent, longitude + window_extent), 
    # 'grid_yt' = (latitude - window_extent, latitude + window_extent)

    # print(f"Coordinates at {TC_timestamp}: {TC_track_coordinates[TC_timestamp]['lon']}, {TC_track_coordinates[TC_timestamp]['lat']}")
    
    # Assign zonal window
    observation_TC_extent[TC_timestamp][grid_xt] = np.arange(TC_track_coordinates[TC_timestamp]['lon'] - observation_TC_window_size,
                                                             TC_track_coordinates[TC_timestamp]['lon'] + observation_TC_window_size + observation_resolution,
                                                             observation_resolution)
    # Assign meridional window
    observation_TC_extent[TC_timestamp][grid_yt] = np.arange(TC_track_coordinates[TC_timestamp]['lat'] - observation_TC_window_size,
                                                             TC_track_coordinates[TC_timestamp]['lat'] + observation_TC_window_size + observation_resolution,
                                                             observation_resolution)
    # Extract GCM data for the given timestamp and spatial extent
    # Notice the modulo on `grid_xt` - this is used to handle Prime Meridian bugs
    observation_TC_container[TC_timestamp] = observation_dataset.sel(time=TC_timestamp)
    observation_TC_container[TC_timestamp] = observation_TC_container[TC_timestamp].sel({grid_xt: observation_TC_extent[TC_timestamp][grid_xt] % 360})
    observation_TC_container[TC_timestamp] = observation_TC_container[TC_timestamp].sel({grid_yt: observation_TC_extent[TC_timestamp][grid_yt]})

    # Interpolate to different resolution (shoot for 0.5 degrees)
    observation_TC_container[TC_timestamp] = observation_TC_container[TC_timestamp].interp(longitude=zonal_basis_vector).interp(latitude=meridional_basis_vector)
    
    return observation_TC_container[TC_timestamp]

In [18]:
def load_observation_TC(TC_track_coordinates: dict,
                        TC_timestamps: list,
                        observation_data: xr.Dataset,
                        observation_resolution: float,
                        target_resolution: float,
                        observation_TC_window_size: int|float,
                        parallel: bool,
                        diagnostic: bool=False):

    ''' Method to load observation data for a given TC given its track coordinates, track timestamps, and reanalysis data. '''
    
    # Get basis vectors for reanalysis data generation
    coarsen_factor = int(np.round(target_resolution / observation_resolution)) # factor by which observation data will be coarsened to match a target resolution
    zonal_basis_vector, meridional_basis_vector = observation_grid_redefinition(TC_track_coordinates,
                                                                                observation_resolution,
                                                                                coarsen_factor,
                                                                                observation_TC_window_size)

    # Initialize a container to hold GCM output connected to each storm timestamp and the corresponding spatial extent
    observation_TC_container = {}
    # Define partial function to streamline function calls, since the only variable argument is `storm_timestamps`
    partial_load_observation_TC_timestamp = functools.partial(load_observation_TC_timestamp,
                                                                 TC_track_coordinates,
                                                                 observation_TC_window_size,
                                                                 observation_resolution,
                                                                 observation_data,
                                                                 zonal_basis_vector,
                                                                 meridional_basis_vector)
    # Keep time for profiling
    start_time = time.time()
    # Initialize container dictionary
    observation_TC_container = {}
    # Iterate over all timestamps to find reanalysis data for the given entry
    for TC_timestamp in TC_timestamps:
        observation_TC_container[TC_timestamp] = partial_load_observation_TC_timestamp(TC_timestamp)
    # Concatenate all GCM output data corresponding to storm into a single xArray Dataset
    observation_TC_data = xr.concat(observation_TC_container.values(), dim='time').sortby('time')

    if diagnostic:
        print(f'Elapsed time to load observation storm: {(time.time() - start_time):.2f} s.')
        print(f'\t per timestamp: {((time.time() - start_time) / len(TC_timestamps)):.2f} s.')

    return observation_TC_data

In [19]:
def observation_TC_generator(dataset_name: str,
                             observation_track_dataset: pd.DataFrame,
                             observation_dataset: xr.Dataset,
                             observation_resolution: float,
                             target_resolution: float,
                             observation_TC_window_size: int,
                             parallel: bool,
                             storm_ID: str|None=None):

    ''' Method to perform all steps related to binding corresponding GFDL QuickTracks and GCM model output together for a given TC. '''

    print(f'[storm_generator] Processing storm ID {storm_ID}...')

    # 4. Find a candidate storm from the track data, ensure it is ordered by time
    TC_track_dataset = TC_tracker.pick_storm(observation_track_dataset, 
                                             selection_method='storm_number', 
                                             storm_ID=storm_ID).sort_values('time')
    # 5. Pull storm-specific timestamps
    TC_track_timestamps = align_timestamps(TC_track_dataset, observation_dataset)
    if len(TC_track_timestamps) == 0:
        print(f"No matching timestamps were found for storm {TC_track_dataset['storm_id'].unique().item()}. Exiting this storm.")
        return
    # 6. Pull storm-specific coordinates to align with track timestamps
    TC_track_coordinates = get_TC_coordinates(TC_track_dataset, 
                                              observation_dataset,
                                              TC_track_timestamps, 
                                              observation_resolution)
    # 7. Load reanalysis data for the iterand storm to align with track data
    observation_TC_dataset = load_observation_TC(TC_track_coordinates, 
                                                 TC_track_timestamps,
                                                 observation_dataset,
                                                 observation_resolution,
                                                 target_resolution,
                                                 observation_TC_window_size,
                                                 parallel=parallel)
    # 8. Append information from track data to object containing reanalysis output.
    observation_TC_dataset = TC_tracker.join_track_GCM_data(storm_track_data=TC_track_dataset,
                                                            storm_gcm_data=observation_TC_dataset,
                                                            storm_time_variable='time')
    # 9. Derive fields present in GCM output data but not directly provided in ERA5 data
    observation_TC_dataset = derive_fields(observation_TC_dataset, dataset_name)
    # 10. Perform adjustments for compatibility with GFDL GCM outputs
    observation_TC_dataset = observation_GFDL_compatibility_adjustments(observation_TC_dataset)
    # 11. Save xArray Dataset to netCDF file
    TC_tracker.save_storm_netcdf(observation_TC_dataset, model_name=dataset_name, experiment_name='OBS')

In [24]:
def main(dataset_name: str,
         date_range: tuple[str, str],
         basin_name: str='global',
         intensity_parameter: str='min_slp',
         intensity_range: tuple[int | float, int | float]=(980, 1000),
         number_of_storms: int=1,
         observation_resolution: float=0.5,
         target_resolution: float=0.5,
         observation_TC_window_size: int|float=12,
         parallel: bool=True):

    # 1. Pull track data for a given date range, basin, and intensity range
    print('Loading TC tracks from IBTrACS.')
    TC_track_dataset = access_IBTrACS_tracks(date_range=date_range, 
                                             basin_name=basin_name, 
                                             intensity_parameter=intensity_parameter, 
                                             intensity_range=intensity_range)
    # 2. Access reanalysis dataset. Ensure this is done lazily to avoid excessive memory usage.
    print(f'Loading observational dataset for {dataset_name}.')
    date_range = [pd.to_datetime(date) for date in date_range]
    observation_dataset = access_observation_data(dataset_name,
                                                  date_range)
    # 2a. Perform observation dataset renaming field correction
    print('Correcting field names.')
    observation_dataset = rename_observation_fields(observation_dataset,
                                                    dataset_name=dataset_name)
    # 2b. Perform observation dataset timestamp adjustments
    print('Adjusting timestamp conventions.')
    observation_dataset = adjust_timestamps(observation_dataset)
    # 2c. Adjust grid to ensure the resolution is equal in both axes
    print('Interpolating grid.')
    observation_dataset = adjust_grid(observation_dataset, observation_resolution)
    # 3. Obtain N randomized storm IDs from the filtered track data, where 'N' is `number_of_storms`
    storm_IDs = TC_tracker.pick_storm_IDs(TC_track_dataset, number_of_storms)
    
    # Define partial function to allow for using Pool.map since `track_data` is equivalent for all subprocesses
    partial_observation_TC_generator = functools.partial(observation_TC_generator, 
                                                         dataset_name,
                                                         TC_track_dataset, 
                                                         observation_dataset,
                                                         observation_resolution,
                                                         target_resolution,
                                                         observation_TC_window_size,
                                                         parallel)
    # Load storms in parallel    
    if parallel:
        with Pool() as pool:
            pool.map(partial_observation_TC_generator, storm_IDs)
            pool.close()
    else:
        for storm_ID in storm_IDs:
            partial_observation_TC_generator(storm_ID)

In [28]:
date_range = ('2019-01-01', '2020-01-01')
basin_name = 'global'
intensity_parameter = 'min_slp'
intensity_range = (980, 1000)
observation_dataset_name = 'CERES'

number_of_storms = 20

main(dataset_name=observation_dataset_name, 
     basin_name=basin_name, 
     date_range=date_range, 
     intensity_parameter=intensity_parameter,
     intensity_range=intensity_range, 
     number_of_storms=number_of_storms,
     parallel=True)

Loading TC tracks from IBTrACS.
Loading observational dataset for CERES.
Correcting field names.
Adjusting timestamp conventions.
Interpolating grid.
[storm_generator] Processing storm ID 2019-218N21088...
[storm_generator] Processing storm ID 2019-242N18126...[storm_generator] Processing storm ID 2019-176N18128...

[storm_generator] Processing storm ID 2019-272N14261...
[storm_generator] Processing storm ID 2019-260N10253...
[storm_generator] Processing storm ID 2019-291N22264...
No matching timestamps were found for storm 2019-291N22264. Exiting this storm.
[storm_generator] Processing storm ID 2019-236N08143...
[storm_generator] Processing storm ID 2019-126S05129...
[save_storm_netcdf] Loading data for TC.model-CERES.experiment-OBS.storm_ID-2019-176N18128.max_wind-21.min_slp-994.basin-WP.nc
[storm_generator] Processing storm ID 2019-001S10162...
[save_storm_netcdf] Loading data for TC.model-CERES.experiment-OBS.storm_ID-2019-242N18126.max_wind-18.min_slp-996.basin-WP.nc
[storm_gener

In [ ]:
dirname = '/projects/GEOCLIM/gr7610/analysis/tc_storage/individual_TCs'
filename = 'TC.model-CERES.experiment-OBS.storm_ID-2019-255N15251.max_wind-59.min_slp-950.basin-EP.nc'
pathname = os.path.join(dirname, filename)
dataset = xr.open_dataset(pathname)
fig, ax = plt.subplots(figsize=(8, 3.5))
dataset['olr'].isel(time=13).plot(ax=ax)
ax.set_aspect('equal')